In [1]:
import torch

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
INTERVALS = 7
# 创建一个从-100到100的浮点数张量作为输入数据
input =torch.arange(-100,100,1, dtype=torch.float32)  # 注意：torch.range已弃用，应使用torch.arange
print("input:",input)

# 获取输入张量的最小值和最大值，用于归一化
global_min = input.min()
global_max = input.max()
print("global_min:",global_min)
print("global_max:",global_max)

# 将输入数据归一化到[0,1]范围内
normalized = (input - global_min) / (global_max - global_min)
print("normalized:",normalized)

# 将归一化后的数据缩放到[0,7]范围，以便进行4位量化（0-7范围可以用3位表示，但使用4位存储）
scaled = normalized * INTERVALS
print("scaled:",scaled)

# 将缩放后的数据裁剪到[0,7]范围并转换为无符号8位整数类型
# 虽然转换为uint8，但实际值仅使用低3位(0-7)
quantized = torch.clamp(scaled, 0, 7).to(torch.uint8)
print("quantized:",quantized)


# 创建一个张量来存储压缩后的数据
# 由于每个字节可以存储两个4位值，所以大小为原始张量的一半（向上取整）
packed = torch.zeros((quantized.numel() + 1) // 2, dtype=torch.uint8, device=quantized.device)
print("packed:",packed)

#### Compress 8bits -> 4 bits
# 将量化后的4位值打包到8位字节中
# 每个字节可以存储两个4位值：低4位存储奇数索引的值，高4位存储偶数索引的值
# 压缩过程：
#   1. 将偶数索引的值左移4位，放入高4位
#   2. 将奇数索引的值直接放入低4位
#   3. 使用按位或运算将两个值合并到一个字节中

# 将偶数索引(0,2,4...)的量化值左移4位放入packed的高4位
packed[:quantized[::2].numel()] = quantized[::2] << 4  

# 如果有奇数个元素，确保处理奇数索引的值
if quantized.numel() > 1:
    # 使用按位或运算将奇数索引(1,3,5...)的量化值放入packed的低4位
    packed[:quantized[1::2].numel()] |= quantized[1::2]

print("packed:",packed)
print("packed.size():",packed.size())  # 显示压缩后的张量大小
print("quantized.size():",quantized.size())  # 显示原始张量大小

### Decompress 4 bits -> 8 bits
# 解压缩过程：从压缩的字节中提取4位值并还原为原始量化值

# 创建一个与原始量化张量相同大小的张量来存储解压缩后的数据
unpacked = torch.zeros(quantized.numel(), dtype=torch.uint8, device=packed.device)

# 计算需要处理的偶数位元素数量
# 取packed.numel()和(unpacked.numel() // 2 + unpacked.numel() % 2)的较小值
# 这确保我们不会超出packed或unpacked的边界
num_even = min(packed.numel(), unpacked.numel() // 2 + unpacked.numel() % 2)

# 从packed的高4位提取数据并还原到unpacked的偶数位置
# 右移4位后与0x0F(00001111)进行按位与操作，确保只保留低4位
unpacked[:2*num_even:2] = (packed[:num_even] >> 4) & 0x0F  

# 计算需要处理的奇数位元素数量
num_odd = min(packed.numel(), unpacked.numel() // 2)

# 从packed的低4位提取数据并还原到unpacked的奇数位置
# 与0x0F(00001111)进行按位与操作，确保只保留低4位
unpacked[1:2*num_odd:2] = packed[:num_odd] & 0x0F          

print("unpacked:",unpacked)
# 验证解压缩是否正确，比较解压缩后的张量与原始量化张量是否相等
print("unpacked allclose to quantized:", torch.allclose(unpacked.float(), quantized.float()))

# 去归一化
unpacked_float = unpacked.to(torch.float)/INTERVALS * (global_max - global_min) + global_min
print("unpacked_float:",unpacked_float)

input: tensor([-100.,  -99.,  -98.,  -97.,  -96.,  -95.,  -94.,  -93.,  -92.,  -91.,
         -90.,  -89.,  -88.,  -87.,  -86.,  -85.,  -84.,  -83.,  -82.,  -81.,
         -80.,  -79.,  -78.,  -77.,  -76.,  -75.,  -74.,  -73.,  -72.,  -71.,
         -70.,  -69.,  -68.,  -67.,  -66.,  -65.,  -64.,  -63.,  -62.,  -61.,
         -60.,  -59.,  -58.,  -57.,  -56.,  -55.,  -54.,  -53.,  -52.,  -51.,
         -50.,  -49.,  -48.,  -47.,  -46.,  -45.,  -44.,  -43.,  -42.,  -41.,
         -40.,  -39.,  -38.,  -37.,  -36.,  -35.,  -34.,  -33.,  -32.,  -31.,
         -30.,  -29.,  -28.,  -27.,  -26.,  -25.,  -24.,  -23.,  -22.,  -21.,
         -20.,  -19.,  -18.,  -17.,  -16.,  -15.,  -14.,  -13.,  -12.,  -11.,
         -10.,   -9.,   -8.,   -7.,   -6.,   -5.,   -4.,   -3.,   -2.,   -1.,
           0.,    1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,    9.,
          10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,   18.,   19.,
          20.,   21.,   22.,   23.,   24.,   25.,   26., 